# Advances in Machine Learning with Big Data

### (part 1 of 2) 
### Trinity 2020 Weeks 1 - 4
### Dr Jeremy Large
#### jeremy.large@economics.ox.ac.uk


&#169; Jeremy Large ; shared under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/)

## 2. Being an econometrician *and* a data scientist

## Contents Weeks 1-4:

1. Introducing this course's dataset

1. **Being an econometrician _and_ a data scientist**

1. Data abundance and 'jaggedness' -> regularization and the problem of overfit

1. Regularization through resampling methods (bootstrap etc.)

1. Regularization through predictor/feature selection (Lasso etc.)

1. Moving from linear regression to the perceptron

1. Moving from linear regression to the random forest (and similar)

In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys, os
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO)

# point at library; I need some lessons on doing good PYTHONPATHs:
REPO_DIR = os.path.dirname(os.getcwd())

UCI_LIB = os.path.join(REPO_DIR, 'lib')
UCI_DATA = os.path.join(REPO_DIR, 'data') 

sys.path.append(UCI_LIB)

UCI_DATA_FILE = os.path.join(UCI_DATA, 'raw.csv') 

from uci_retail_data import stock_codes, uci_files 

Populating the interactive namespace from numpy and matplotlib


### Pull in and prepare our data

In [2]:
if os.path.exists(UCI_DATA_FILE):
    df = uci_files.load_uci_file(UCI_DATA_FILE, uci_files.SHEET_NAME)
else:
    df = uci_files.load_uci_file(uci_files.REMOTE_FILE, uci_files.SHEET_NAME)
    df.to_csv(UCI_DATA_FILE)
    logging.info('Saving a copy to ' + UCI_DATA_FILE)

2020-04-10 10:47:37,702 INFO:Loading C:\Users\jerem\Documents\work\Oxford\SBS\MLBD\ox-sbs-ml-bd\data\raw.csv , sheet Year 2009-2010
2020-04-10 10:47:40,419 INFO:Loaded C:\Users\jerem\Documents\work\Oxford\SBS\MLBD\ox-sbs-ml-bd\data\raw.csv , sheet number one, obviously


Clean data:

In [3]:
# Here, I call the irrelevant lines 'invalids':
invalids = stock_codes.invalid_series(df)

Aggregate into invoices:

In [4]:
invoices = stock_codes.invoice_df(df, invalid_series=invalids)

In [5]:
invoices.head(2)

,customer,codes_in_invoice,items_in_invoice,invoice_spend,hour,month,words,country,words_per_item
Invoice,,,,,,,,,
489434,13085.0,8,166,505.3,7,200912,"{DOUGHNUT, FONT, SAVE, BOX, RECORD, 7"", SIZE, ...",United Kingdom,3.625
489435,13085.0,4,60,145.8,7,200912,"{LARGE, FAIRY, BOWL, WITH, HEART, DOG, DESIGN,...",United Kingdom,4.000


In [6]:
invoices.tail(2)

,customer,codes_in_invoice,items_in_invoice,invoice_spend,hour,month,words,country,words_per_item
Invoice,,,,,,,,,
538170,13969.0,25,133,317.59,19,201012,"{BLACK, LIGHTS, LARGE, FANNY'S, HOUSE, VINTAGE...",United Kingdom,2.92
538171,17530.0,65,194,300.64,20,201012,"{MORE, PLAYHOUSE, HEARTS, MAKE, PLACE, DECORAT...",United Kingdom,2.20


In [7]:
invoices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20577 entries, 489434 to 538171
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer          18969 non-null  float64
 1   codes_in_invoice  20577 non-null  int64  
 2   items_in_invoice  20577 non-null  int64  
 3   invoice_spend     20577 non-null  float64
 4   hour              20577 non-null  int64  
 5   month             20577 non-null  int64  
 6   words             20577 non-null  object 
 7   country           20577 non-null  object 
 8   words_per_item    20577 non-null  float64
dtypes: float64(3), int64(4), object(2)
memory usage: 1.6+ MB


### Set a prediction problem:

**Given the time, date, and complexity of an invoice, what's its expected spend?**

First, we'll attack this in a familiar way: using, as best we can, *linear regression*. 

*  However, we'll take time to compare two suitable `python` libraries for this.

### 1. [`statsmodels`](https://www.statsmodels.org/stable/index.html)

* package for established statistics; has [some of the feel](https://www.statsmodels.org/stable/example_formulas.html) of R

* *statsmodels: Econometric and statistical modeling with python* Seabold, Skipper, and Josef Perktold. Proceedings of the 9th Python in Science Conference, 2010.

* funded at [Google Summer of Code (GSOC) 2009-2017](https://summerofcode.withgoogle.com/) and by hedge fund [AQR](https://www.aqr.com)      

### 2. [`scikit-learn`](https://scikit-learn.org/stable/index.html)

* package for machine learning

* *Scikit-learn: Machine Learning in Python*, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

They both do linear regression.

#### Lets import statsmodels / scikit-learn and packages that they use:

In [8]:
import numpy as np    # fast handling of matrices of real numbers, and similar

import pandas as pd     # great tool for wielding and combining rectangular datasets with varied, named, columns



import sklearn

import statsmodels

In [8]:
# in both cases, we will have to import specifically the bits of the package that we need, for example:
import statsmodels.formula.api as smf

#### Reminder of linear regression:

We have an i.i.d. sequence of observations, $\{(y_i, x_i), i=0, 1, ...\}$ where we are interested in moments of the R.V. $y_i$, conditional on the multivariate R.V.  $x_i$ (of length, say, $p$). 

We *postulate* a linear relationship of the following form:

\begin{equation}
y_i = x_i ' \beta + \epsilon_i,
\end{equation}
where $\beta$ is a vector of parameters of length $p$, and the i.i.d. sequence of random variables $\{\epsilon_i\}$ is independent of the regressors $\{x_i\}$

#### A personal story:

Now, what do I mean here by the word: *'postulate'*? 

For the econometrician in me, it means I *sincerely assert*.

For the machine learner in me, I'm not sure that's quite it. It's more that - pragmatically - I entertain this model.

* an important part of this course will be to flesh-out this comparison

#### 1. Fitting a linear regression with `statsmodels`

Describe the model in natural statistician's terms:

In [9]:
formula_string = 'np.log(invoice_spend) ~ np.log(codes_in_invoice) + np.log(items_in_invoice) + hour + month + words_per_item'

Specify that the model is OLS:

In [10]:
lm1 = smf.ols(formula=formula_string, data=invoices)

Common, pythonic, step, next, namely to `fit()`:

In [11]:
fit = lm1.fit()

In [12]:
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(invoice_spend)   R-squared:                       0.815
Model:                               OLS   Adj. R-squared:                  0.815
Method:                    Least Squares   F-statistic:                 1.813e+04
Date:                   Fri, 10 Apr 2020   Prob (F-statistic):               0.00
Time:                           10:50:04   Log-Likelihood:                -16469.
No. Observations:                  20577   AIC:                         3.295e+04
Df Residuals:                      20571   BIC:                         3.300e+04
Df Model:                              5                                         
Covariance Type:               nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   29.2646     29.005      1.009      0.313     -27.587      86.116
np.log(codes_in_invoice)     0.1440      0.005     31.038      0.000       0.135       0.153
np.log(items_in_invoice)     0.7087      0.003    215.811      0.000       0.702       0.715
hour                        -0.0122      0.002     -7.733      0.000      -0.015      -0.009
month                       -0.0001      0.000     -0.945      0.345      -0.000       0.000
words_per_item               0.0168      0.006      2.691      0.007       0.005       0.029
==============================================================================
Omnibus:                     2692.737   Durbin-Watson:                   1.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19904.742
Skew:                           0.405   Prob(JB):                         0.00
Kurtosis:                       7.750   Cond. No.                     1.55e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.55e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The 'condition number' refers to $X'X$ -- and indicates how stably we can expect to invert that matrix

#### Comments on linear regression with `statsmodels`

* Thorough implementation of linear regression, elegant presentation, including tests:
    * significance tests (t-stats, F-tests, p-values, ...)
    * specification tests

* Imagine these tests pass (though they don't here)
    * we may conclude that we now *know* the Data Generating Process
       * our parametric modelling has been successful
    * we understand statistical aspects of the mechanisms out there
    * -> we are in a position to offer *explanations* of past, present and future observation
    

* All the data has been used in fitting the regression
    * like e.g. MLEs, resulting estimates are statistically *efficient*.
    * But, to use up all the data like that:
      * was that greedy?
      * was it prudent?

#### 2. Fitting a linear regression with `scikit-learn`

For `scikit-learn`, we'll have to build some additional columns in our dataframe:

In [13]:
invoices['hour_sqd'] = invoices.hour**2
invoices['log_inv_spend'] = np.log(invoices.invoice_spend)
invoices['log_n_codes'] = np.log(invoices.codes_in_invoice)
invoices['log_n_items'] = np.log(invoices.items_in_invoice)

In [14]:
# Reminder of the formula:
formula_string

'np.log(invoice_spend) ~ np.log(codes_in_invoice) + np.log(items_in_invoice) + hour + month + words_per_item'

In [15]:
# Now set out `y` and `X` variables:
y = pd.DataFrame(invoices['log_inv_spend'])
X = invoices[['log_n_codes', 'log_n_items', 'hour', 'month', 'words_per_item']] # , 'hour_sqd'

In [16]:
#  pull in the libraries:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [17]:
reg_all = LinearRegression()
reg_all.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [18]:
print('intercept is ', reg_all.intercept_)
print('coefficients are ', reg_all.coef_)
print('R2 is ', reg_all.score(X, y))

intercept is  [29.26457648]
coefficients are  [[ 1.43967271e-01  7.08724911e-01 -1.22246009e-02 -1.36293685e-04
   1.67567537e-02]]
R2 is  0.8150405334343745


In [19]:
reg = LinearRegression()
scores = []

In [20]:
from sklearn import inspection
from sklearn.model_selection import KFold

In [21]:
#  
kfold = KFold(n_splits=10, shuffle=True)

In [22]:
for i, (train, test) in enumerate(kfold.split(X, y)):
    reg.fit(X.iloc[train], y.iloc[train])
    score = reg.score(X.iloc[test], y.iloc[test])
    pred = reg.predict(X.iloc[test])
    r2 = metrics.r2_score(y.iloc[test], pred)
    scores.append(r2)

In [23]:
plt.plot(scores)
plt.xaxis("different splits of "")
plt.show()

SyntaxError: invalid syntax (<ipython-input-23-eb92a3c8d270>, line 2)

In [ ]:
reg = LinearRegression()